okt하는 함수를 따로 선언하지않고 피클쓰는방식
: 먼저 정제하고 토큰화한걸 다시 문장으로만들어서 
속성없이 tfidf벡터라이저에 넣기

test2_nofunc_vectorizer.pkl로 저장했따.


In [1]:
# !pip install konlpy

In [1]:
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
import tensorflow as tf
import numpy as np

#1. Naver Sentiment Movie Corpus v1.0 다운로드
train_file = tf.keras.utils.get_file('train.txt', 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt')
# test_file = tf.keras.utils.get_file( 'test.txt',  'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt')

#파일을 이진모드로 읽어온다. 디코드는 utf8로한다.
train_text = open(train_file,'rb').read().decode(encoding='utf-8')
# test_text = open(test_file,'rb').read().decode(encoding='utf-8')


In [2]:
import pandas as pd

df_train = pd.DataFrame({
    'feature':[row.split('\t')[1] for row in train_text.split('\n')[1:] if row.count('\t')>0],
    'target': [ int(row.split('\t')[2]) for row in train_text.split('\n')[1:] if row.count('\t')>0]
})

#데이터 크기 지정
df_train = df_train[::6]

In [3]:
#문장 추출
texts= [ ]
for i in df_train['feature']:
    texts.append(i)
len(texts)

25000

In [4]:
#문자열이 아닌게 있는지 확인
for i in texts:
    if type(i)!=str:
        print(i)

In [5]:
# train 데이터 입력값(X)을 정제(Cleaning)
import re
from soynlp.normalizer import repeat_normalize

def clean_korean_text(text):
    # 특수 문자 및 숫자 제거
    text = re.sub(r'[^가-힣ㄱ-ㅎㅏ-ㅣ\s]', '', text)
    # 반복되는 자음, 모음 제거 (e.g., 'ㅋㅋㅋ' -> 'ㅋ')
    text = repeat_normalize(text, num_repeats=1)
    # 띄어쓰기 정규화 (연속된 공백 문자를 하나의 공백 문자로 변환)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

clean_texts=[]
for i in texts:
    clean_texts.append(clean_korean_text(i))

In [7]:
len(clean_texts)
# clean_texts[:5]

25000

In [8]:
#정제된 텍스트를 벡터화하기전에 토큰화한다
from konlpy.tag import Okt
tokenized_clean_texts =[ Okt().morphs(i) for i in clean_texts ]
tokenized_clean_texts

[['아', '더빙', '진짜', '짜증나네요', '목소리'],
 ['원작', '의', '긴장감', '을', '제대로', '살려내지못', '했다'],
 ['울면',
  '서',
  '손',
  '들',
  '고',
  '횡단보도',
  '건널',
  '때',
  '뛰',
  '쳐나',
  '올',
  '뻔',
  '이범수',
  '연기',
  '드럽게',
  '못',
  '해'],
 ['이건',
  '정말',
  '깨알',
  '캐스팅',
  '과',
  '질퍽',
  '하',
  '지',
  '않은',
  '산뜻한',
  '내',
  '용구성',
  '이',
  '잘',
  '버무러진',
  '깨알',
  '일드'],
 ['주제', '는', '좋은데', '중반', '부터', '지루하다'],
 ['엄포스',
  '의',
  '위력',
  '을',
  '다시',
  '한번',
  '깨닫게',
  '해준',
  '적남',
  '꽃',
  '검사',
  '님',
  '도',
  '연기',
  '정말',
  '좋았어요',
  '완전',
  '명품',
  '드라마'],
 ['키이라', '나이틀리', '가', '연기', '하고자', '했던건', '대체', '정신장애', '일까', '틱장애', '일까'],
 ['오게', '두어라', '서리', '한', '이', '굶주렸다'],
 ['난', '우리', '영화', '를', '사랑', '합니다'],
 ['재미있어요'],
 ['보는내내', '그대로', '들어맞는', '예측', '카리스마', '없는', '악역'],
 ['예전',
  '작품',
  '캐릭터',
  '에피소드',
  '재탕',
  '삼',
  '탕',
  '사골',
  '우려',
  '먹듯',
  '우리',
  '고',
  '내용',
  '은',
  '산',
  '으로',
  '가고',
  '시청률',
  '은',
  '아예',
  '안',
  '나오고',
  '이제',
  '회',
  '중반',
  '인데',
  '부작',
  '이라니'],
 ['졸작'],
 ['한국

In [9]:
len(tokenized_clean_texts)

25000

In [10]:
#이미 토큰화된걸 ' ' 공백한칸을 기준으로 다시 합쳐준다.
rejoined_tokenized_texts = [' '.join(i) for i in tokenized_clean_texts]
rejoined_tokenized_texts

['아 더빙 진짜 짜증나네요 목소리',
 '원작 의 긴장감 을 제대로 살려내지못 했다',
 '울면 서 손 들 고 횡단보도 건널 때 뛰 쳐나 올 뻔 이범수 연기 드럽게 못 해',
 '이건 정말 깨알 캐스팅 과 질퍽 하 지 않은 산뜻한 내 용구성 이 잘 버무러진 깨알 일드',
 '주제 는 좋은데 중반 부터 지루하다',
 '엄포스 의 위력 을 다시 한번 깨닫게 해준 적남 꽃 검사 님 도 연기 정말 좋았어요 완전 명품 드라마',
 '키이라 나이틀리 가 연기 하고자 했던건 대체 정신장애 일까 틱장애 일까',
 '오게 두어라 서리 한 이 굶주렸다',
 '난 우리 영화 를 사랑 합니다',
 '재미있어요',
 '보는내내 그대로 들어맞는 예측 카리스마 없는 악역',
 '예전 작품 캐릭터 에피소드 재탕 삼 탕 사골 우려 먹듯 우리 고 내용 은 산 으로 가고 시청률 은 아예 안 나오고 이제 회 중반 인데 부작 이라니',
 '졸작',
 '한국영 화 흥행 코드 갈등 갈등 계 에 속 갈등 화해 감동 평점 점 남발 흥행 뻔하지 뭐',
 '정말 쓰레기 영화 입니다',
 '성룡 영화 중 최악 인 듯 ㅋㅋ',
 '재미있는 영화 입니다',
 '잔잔한게 생각 보다 볼 만 한 영화 인거 같습니다 ㅋ',
 '재미있게 봤습니다 매력 적 인 행복 이네 요 ㅎㅎㅎ',
 '시청률 기준 이 되는 패널 가구 들 머 하는거지 명작 드라마 다 망치 네 ㅡㅡ 내 가 다 서운하다',
 '중국인 특유 의 과장 허풍 있어 보이려고 안간힘 쓴 노력 은 가상하나 고증 과 현 실감 떨어지는 설정 이 거북 스럽다 도대체 그 들 은 왜 이렇게 까지 스스로 를 과대 포장 하 는 것 인지',
 '아니 이 게 왜 점 대 일까',
 '아주 모자라진 않다',
 '다르 덴 이냐 리투 차이밍량 이 하나 도 안 섞인 채 짬뽕 그릇 에 담겨있다',
 '재미있다고 허풍 떨지 마세요',
 '그만 좀 끌 고 이제 끝내 라 지겹다 지겨워',
 '배우 들 은 지네 들 안 뜨니까 이런 영화 찍을껀데 왜 안 뜨는지 진짜 모르는거야 면상 딱 보면 알겠구만 왜

In [11]:
len(rejoined_tokenized_texts)

25000

In [12]:
#토큰화하고 다시합쳐진 문장을 fit_transform 해준다.
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle

vectorizer=TfidfVectorizer( )
X = vectorizer.fit_transform(rejoined_tokenized_texts)

# vectorizer 객체를 저장
with open('test2_nofunc_vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)
    
# y = df_train.iloc[:,1]


In [13]:
import pickle

# 저장된 vectorizer 객체를 불러옵니다.
with open('test2_nofunc_vectorizer.pkl', 'rb') as f:
    loaded_test_vectorizer = pickle.load(f)

#사용자가 입력하는 데이터에 transform 시험 

In [18]:
#만약 이런 카톡대화가 들어온다면
test_texts = ['이거 정말 잼따. ', '아니 무슨 이딴게 다있냐;;','ㅋzz ㅋㅋ ㅋㅋㅋㅋ']

#문자열이 아닌게 있는지 확인
for i in test_texts:
    if type(i)!=str:
        print(i)
        
# train 데이터 입력값(X)을 정제(Cleaning)
import re
from soynlp.normalizer import repeat_normalize

def clean_korean_text(text):
    # 특수 문자 및 숫자 제거
    text = re.sub(r'[^가-힣ㄱ-ㅎㅏ-ㅣ\s]', '', text)

    # 반복되는 자음, 모음 제거 (e.g., 'ㅋㅋㅋ' -> 'ㅋ')
    text = repeat_normalize(text, num_repeats=1)

    # 띄어쓰기 정규화 (연속된 공백 문자를 하나의 공백 문자로 변환)
    text = re.sub(r'\s+', ' ', text).strip()

    return text

clean_test_texts=[]
for i in test_texts:
    clean_test_texts.append(clean_korean_text(i))
    
#정제된 텍스트를 벡터화하기전에 토큰화한다
from konlpy.tag import Okt
tokenized_clean_test_texts =[ Okt().morphs(i) for i in clean_test_texts ]
tokenized_clean_test_texts

#토큰화된걸 ' ' 공백한칸을 기준으로 다시 합쳐준다.
rejoined_tokenized_test_texts = [' '.join(i) for i in tokenized_clean_test_texts]
rejoined_tokenized_test_texts

# 다시합쳐진 토큰화 문장을 transform 해준다.
X_test_sample = loaded_test_vectorizer.transform(rejoined_tokenized_test_texts)
X_test_sample_dense = X_test_sample.toarray()
print(X_test_sample_dense)
print(len(X_test_sample_dense[0]))

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
34583
